In [1]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead

In [2]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

In [3]:
model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

In [4]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode('Я люблю гулять со своей собакой', return_tensors='pt')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = model.generate(input_ids, max_length=50).to('cuda')

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Output:
----------------------------------------------------------------------------------------------------
Я люблю гулять со своей собакой.

— А я люблю гулять с тобой.

— А я люблю гулять с тобой.

— А я люблю гулять с тобой.

— А я люблю гулять с тобой


In [5]:
# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,  
    max_length=50, 
    num_beams=5, 
    early_stopping=True
).to('cuda')

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
----------------------------------------------------------------------------------------------------
Я люблю гулять со своей собакой.













































In [6]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    early_stopping=True
).to('cuda')

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
----------------------------------------------------------------------------------------------------
Я люблю гулять со своей собакой.

— Я тоже, — сказала она, и в ее голосе прозвучала нотка грусти, но она тут же взяла себя в руки и добавила: — Но я не хочу, чтобы ты уходил


In [7]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True
).to('cuda')

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
----------------------------------------------------------------------------------------------------
0: Я люблю гулять со своей собакой.

— Я тоже, — сказала она, и в ее голосе прозвучала нотка грусти, но она тут же взяла себя в руки и добавила: — Но я не хочу, чтобы ты уходил
1: Я люблю гулять со своей собакой.

— Я тоже, — сказала она, и в ее голосе прозвучала нотка грусти, но она тут же взяла себя в руки и добавила: — Но я не хочу, чтобы ты гу
2: Я люблю гулять со своей собакой.

— Я тоже, — сказала она, и в ее голосе прозвучала нотка грусти, но она тут же взяла себя в руки и добавила: — Но я не могу гулять с собакой,
3: Я люблю гулять со своей собакой.

— Я тоже, — сказала она, и в ее голосе прозвучала нотка грусти, но она тут же взяла себя в руки и добавила: — Но я не могу гулять одна. Я
4: Я люблю гулять со своей собакой.

— Я тоже, — сказала она, и в ее голосе прозвучала нотка грусти, но она тут же взяла себя в руки и добавила: — Но я не хочу, чтобы ты уходила


In [9]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.manual_seed(0)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=0
).to('cuda')

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
----------------------------------------------------------------------------------------------------
Я люблю гулять со своей собакой Сонечка у меня большая очень умная такая почему-то я не хочу снова ревновать ее когда она рядом с нами она сразу спокойней
это вопрос к психологу. перепробуйте успокоительные, но не вин


In [10]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.manual_seed(0)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=0, 
    temperature=0.7
).to('cuda')

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
----------------------------------------------------------------------------------------------------
Я люблю гулять со своей собакой Сонечка у меня большая и добрая, она никогда не лает на прохожих, а только когда я прошу ее об этом, она всегда выполняет все, что я прошу.  Мы с ней гуляем, играем


In [11]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.manual_seed(0)

# set top_k to 50
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=50
).to('cuda')

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
----------------------------------------------------------------------------------------------------
Я люблю гулять со своей собакой Сонечка у меня большая очень умная такая очень добрая.  Я ее так называю, потому что она очень похожа на человека.  Может все, что я делаю, делать так, как и человек, а может


In [12]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.manual_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_p=0.92, 
    top_k=0
).to('cuda')

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
----------------------------------------------------------------------------------------------------
Я люблю гулять со своей собакой Сонечкой по набережной Москвы-реки, почему-то я попадаю под покровительством совершенно других знаков зодиака, хотя до той любви все же сто лет.&nbsp; Вхожу в воду, а иду


In [13]:
# set seed to reproduce results. Feel free to change the seed though to get different results
torch.manual_seed(0)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=3
).to('cuda')

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Output:
----------------------------------------------------------------------------------------------------
0: Я люблю гулять со своей собакой». Это очень мило звучит, но если человек не любит гулять или он просто хочет быть в тени, собака становится в его глазах слабым звеном. У животного также может сформироваться определенное мнение о нем или самом себе.

1: Я люблю гулять со своей собакой.


29008018	ja4menevo	2012-10-12 01:07:00	Читаю, как всё-таки правильно пишется на разных языках - "ш
2: Я люблю гулять со своей собакой
Герберт Закс




Герберт Закс

Я люблю гулять со своей собакой





1. 

ОПАСНОСТЬ 


На следующий
